# Guide to Orthomapping (Part 3)

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Continued-from-Part-2" data-toc-modified-id="Continued-from-Part-2-1">Continued from Part 2</a></span></li><li><span><a href="#Step-4.-Managing-imagery" data-toc-modified-id="Step-4.-Managing-imagery-2">Step 4. Managing imagery</a></span><ul class="toc-item"><li><span><a href="#4.1-Get-or-set-image-collection-processing-status" data-toc-modified-id="4.1-Get-or-set-image-collection-processing-status-2.1">4.1 Get or set image collection processing status</a></span></li><li><span><a href="#4.2-Add-images-to-image-collection" data-toc-modified-id="4.2-Add-images-to-image-collection-2.2">4.2 Add images to image collection</a></span></li><li><span><a href="#4.3-Delete-images-from-image-collection" data-toc-modified-id="4.3-Delete-images-from-image-collection-2.3">4.3 Delete images from image collection</a></span></li><li><span><a href="#4.4-Reset-image-collection" data-toc-modified-id="4.4-Reset-image-collection-2.4">4.4 Reset image collection</a></span></li><li><span><a href="#4.5-Delete-image-collection" data-toc-modified-id="4.5-Delete-image-collection-2.5">4.5 Delete image collection</a></span></li></ul></li><li><span><a href="#Conclusion" data-toc-modified-id="Conclusion-3">Conclusion</a></span></li><li><span><a href="#References" data-toc-modified-id="References-4">References</a></span></li></ul></div>

## Continued from Part 2


Previously, <a href="orthomapping_guide_1.ipynb">Part 1</a> has introduced the concept of Orthomapping, the pre-requisite configuration environment for orthomapping tools to run on, and how to organize imageries and create imagery collection layer item, and <a href="orthomapping_guide_2.ipynb">Part 2</a> of this guide has covered how to apply block adjustments, manipulate control points, and compute seamlines and color correction, and procedures in generating orthomosaic, DEM, and Vegetation Indices through orthomapping tool. 

Next, in Part 3, we will talk about how to handle the processed imageries, which includes how to add/delete images in the image collection, and reset/delete the image collection. 

Let's get started! First, get connected to your GIS instance.

In [1]:
import arcgis
from arcgis.gis import GIS

portal_url = "https://yourportal.domain.com/portal"
portal_username = "your username"
portal_password = "your password"

gis = GIS(url=portal_url, username=portal_username, password=portal_password)

The `Image Collection Layer` item has already been created in <a href="orthomapping_guide_1.ipynb">Part 1</a>, so let us reuse the reference to the layer item here.

In [3]:
image_collection_item = gis.content.search("omImageCollection")[0]
image_collection_item.url

'https://yourportal.domain.com/server/rest/services/Hosted/omImageCollection/ImageServer'

## Step 4. Managing imagery

### 4.1 Get or set image collection processing status

__[arcgis.raster.orthomapping.get_processing_states()](https://developers.arcgis.com/python/api-reference/arcgis.raster.orthomapping.html#get-processing-states)__ and __[arcgis.raster.orthomapping.alter_processing_states()](https://developers.arcgis.com/python/api-reference/arcgis.raster.orthomapping.html#alter-processing-states)__ are used to query and change processing states. The states are normally set based on orthomapping processes performed on image collection. It is important as it will determine how the data will be handled in the further orthomapping workflows.

In [45]:
from arcgis.raster.orthomapping import get_processing_states
get_processing_states(image_collection=image_collection_item)

{'blockadjustment': 'full',
 'dem': {'match_mode': 'ETM',
  'object_size': 15,
  'ground_spacing': None,
  'minimum_pairs': 8,
  'minimum_adjustment_quality': 0.2,
  'maximum_diff_gsd': 2,
  'maximum_diff_OP': 8,
  'interpolation_method': 'TRIANGULATION',
  'smooth_method': 'GAUSS5X5',
  'surface_type': 'DSM'},
 'seamlines': {'computation_method': 'DISPARITY',
  'blend_width': None,
  'blend_type': 'BOTH',
  'request_size': 1000,
  'request_size_type': 'PIXELS',
  'blend_width_units': 'PIXELS',
  'min_region_size': 100,
  'min_thinness_ratio': 0.05,
  'max_sliver_size': 20},
 'colorcorrection': {'balancing_method': 'DODGING',
  'surface_type': 'SINGLE_COLOR'},
 'adjust_index': 2,
 'imagetype': 'UAV/UAS'}

In [46]:
from arcgis.raster.orthomapping import alter_processing_states

new_states ={'blockadjustment': 'quick',
             'dem': '',
             'seamlines':{'computation_method': 'DISPARITY',
                          'blend_width': None,
                          'blend_type': 'BOTH',
                          'request_size': 1000,
                          'request_size_type': 'PIXELS',
                          'blend_width_units': 'PIXELS',
                          'min_region_size': 100,
                          'min_thinness_ratio': 0.05,
                          'max_sliver_size': 20},
             'colorcorrection': {'balancing_method': 'DODGING',
                                 'surface_type': 'SINGLE_COLOR'},
             'adjust_index': 1,
             'imagetype': 'UAV/UAS'}

alter_processing_states(image_collection=image_collection_item, new_states=new_states)

{'orthomapping': {'blockadjustment': 'quick',
  'dem': '',
  'seamlines': {'computation_method': 'DISPARITY',
   'blend_width': None,
   'blend_type': 'BOTH',
   'request_size': 1000,
   'request_size_type': 'PIXELS',
   'blend_width_units': 'PIXELS',
   'min_region_size': 100,
   'min_thinness_ratio': 0.05,
   'max_sliver_size': 20},
  'colorcorrection': {'balancing_method': 'DODGING',
   'surface_type': 'SINGLE_COLOR'},
  'adjust_index': 1,
  'imagetype': 'UAV/UAS'}}

### 4.2 Add images to image collection

__[arcgis.raster.analytics.add_image()](https://developers.arcgis.com/python/api-reference/arcgis.raster.analytics.html#add-image)__ can help add new images to an exisiting image collection. This will not change anything else associated with other images, e.g., existing seamlines, footprints, control points, etc.

In [47]:
from arcgis.raster.analytics import add_image

In [48]:
image_item = gis.content.search("YUN_0040.JPG")[0]

In [49]:
raster_type_params = {
    "gps":[["YUN_0040.JPG",34.006989,-117.09279,725.13]], "isAltitudeFlightHeight":"False",
    "cameraProperties":{"maker":"Yuneec", "model":"E90", "focallength":8,"columns":5472, "rows":3648, "pixelsize":0.0024},
    "averagezdem":{
        "url":"https://elevation3d.arcgis.com/arcgis/rest/services/WorldElevation3D/Terrain3D/ImageServer"
    }
}

In [1]:
add_image(image_collection=image_collection_item, input_rasters=[image_item], raster_type_name="UAV/UAS", 
          raster_type_params=raster_type_params)

'https://yourportal.domain.com/server/rest/services/Hosted/omImageCollection/ImageServer'

### 4.3 Delete images from image collection 

__[arcgis.raster.analytics.delete_image()](https://developers.arcgis.com/python/api-reference/arcgis.raster.analytics.html#delete-image)__ can help remove existing images from an image collection. This will not change anything else associated with other images, e.g., existing seamlines, footprints, control points, etc.

In [51]:
from arcgis.raster.analytics import delete_image

In [2]:
delete_image(image_collection=image_collection_item, where="objectID=5")

'https://yourportal.domain.com/server/rest/services/Hosted/omImageCollection/ImageServer'

### 4.4 Reset image collection

__[arcgis.raster.orthomapping.reset_image_collection()](https://developers.arcgis.com/python/api-reference/arcgis.raster.orthomapping.html#reset-image-collection)__ can help reset an image collection imagery layer item to its original state. 

It will reset the block adjustment results that are already computed for the images, rebuild footprints, reset seamlines, and remove color correction results. 

It will not remove the existing control points feature class, flight path feature class, etc, that have been calculated from previous block adjustment run.

It will not remove the derived products from adjusted image collection.

In [53]:
from arcgis.raster.orthomapping import reset_image_collection

In [54]:
reset_image_collection(image_collection=image_collection_item)

True

### 4.5 Delete image collection

__[arcgis.raster.analytics.delete_image_collection()](https://developers.arcgis.com/python/api-reference/arcgis.raster.analytics.html#delete-image-collection)__ can help delete 
an image collection imagery layer item from our ArcGIS Enterprise. 

All other associated results and products that are not stored together with image collection will remain. 

In [55]:
from arcgis.raster.analytics import delete_image_collection

In [56]:
delete_image_collection(image_collection=image_collection_item)

True

## Conclusion

In this guide, we have gone through a total of four steps in handling imageries, and generating orthomapping products, namely:

 - __Step 1. Getting Organized__, in which we will discuss how to create image collection layer item from multiple images;
 - __Step 2. Getting Adjusted__, that will introduce how to apply block adjustments, manipulate control points, and compute seamlines and color correction.
 - __Step 3. Getting Results__, in which we will talk about procedures in generating orthomosaic, DEM, and Vegetation Indices through orthomapping tools.
 - __Step 4. Managing Imageries__, that will demonstrate how to add/delete images in the image collection, and reset/delete the image collection.

Now with orthomapping products such as a DTM or DSM, and an orthomosaic with aerial digital imagery, etc. being obtained using Python API methods, you can go on to do all kinds of analysis with elevation data, or use your orthomosaic map for applications in emergency response, resource management, real estate, and more <a href='#References'>[1]</a>.

## References

<a href="#Conclusion">[1]</a> Lenhardt, Ortho Mapping in ArcGIS Pro Part III: Getting Results,https://www.esri.com/arcgis-blog/products/arcgis-pro/imagery/ortho-mapping-products-arcgis-pro/ [Online] [Assessed on August 13, 2019]